Bank Of Israel (BOI) Datasets
=============================

Datasets available at http://www.boi.org.il/ under Time Series.
There is a large amount of datasets there, but unfortunately it's not freely given in a open format easily, This is an attempt to liberate it for easier use

In [29]:
# This page was obtained by a silly url-hack on the search, simply increased the pagination size.
# Took two pages in 800 entry pagination, just sliced them together.
# http://www.boi.org.il/en/DataAndStatistics/_layouts/boi/handlers/WebPartHandler.aspx?wp=SeriesSearch&lang=en-US&PaggingSize=800&textOrCode=&frequency=0&priceBase=0&season=0&economicSector=0&range=0&attach=0&interestType=0&investmentType=0&webUrl=%2Fen%2FDataAndStatistics&page=2&_=1483723339736


from lxml import etree
from io import StringIO
import re

tmp_html = open(r'joined_dataset_list.html','rb').read()
locs = re.findall("Serie Code(.*?)</div>", tmp_html.decode('utf-8') , re.DOTALL)


# Series Code, Data Source, Data Type, Process, Documentation
headers = ["SeriesCode", "DataSource", "DataType", "Process", "Documentation"]
dataset = list(map(re.compile('\n\ *(.*?)</p>').findall, locs))


In [42]:
import pandas as pd

df = pd.DataFrame(dataset, columns=headers)
open('boi_datasets.json','wb').write(df.to_json().encode('utf-8'))


251859

In [63]:
import requests
import concurrent
from tqdm import tqdm
# This takes about 5 Minutes on the 1455 i recovered
for idx, row in tqdm(list(df.iterrows())):
    res = requests.get("http://www.boi.org.il/en/DataAndStatistics/Pages/boi.ashx?Command=DownloadSeriesExcel&SeriesCode={id}|&DateStart=01/03/1975&DateEnd=30/09/2016&Level=3|&Sid=66|".format(id=row['SeriesCode'])).text
    open('data\\{id}.xls'.format(id=row.SeriesCode),'wb').write(res.encode('utf-8'))

100%|██████████████████████████████████████████████████████████████████████████████| 1455/1455 [06:10<00:00,  4.17it/s]


In [85]:
import datetime
import zipfile
import os
now = datetime.datetime.now()
zf = zipfile.ZipFile(now.strftime("data/boi-dataset-%d.%m.%y.zip"),'w', compression=zipfile.ZIP_LZMA)
[zf.write('data\\' + f,arcname=f) for f in os.listdir("data\\") if f.endswith('xls')]
zf.close()



In [205]:
desc, data = pd.read_html("data\\TSB_BAGR_MAKAM_05M.D.xls")
def parse_data(desc, data):
    desc.columns = ["id", "value"]        
    desc.columns = ['id', 'value']
    desc = desc.set_index('id')['value'].to_dict()
    data.columns = ["Date", "Value"]

    data['Date'] = pd.to_datetime(data["Date"], format="%d/%m/%Y")
    return desc,data
    
def get_html_data(filename):
    return parse_data(*pd.read_html("data\\" + filename))

# Plot all Daily Graphs

In [216]:

from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.io import push_notebook
from bokeh.charts import Chart

output_notebook()

for x in [x for x in os.listdir('data//') if '.D.' in x]:
    try:
        desc, data = get_html_data(x) 
    except:
        # RIB_ROI.D.xls is bugged for some reason
        print("Lost on " + x)
        continue
    c = figure(title=desc['Documentation'], plot_height=400, plot_width=800,x_axis_type="datetime")
    c.line(data['Date'],data['Value'],color='navy', alpha=0.5, legend=desc['Description'])
    push_notebook()
    show(c, notebook_handle=True)


Loading BokehJS ...

Lost on RIB_BOI.D.xls


In [224]:
# Makam Lines
from itertools import cycle
cols = cycle(['red', 'blue', 'orange', 'navy'])
line_dash = cycle([[2,2],[1,1],[3,3]])
line_width = cycle([2,3,4])
mkm = figure(title="Makam - Various Maturities", plot_height=700, plot_width=950, x_axis_type="datetime")
# Only bring in a third of them, otherwise the graph is messy
for x in [x for x in os.listdir('data//') if '.D.' in x and 'MAKAM' in x][::3]:
        desc, data = get_html_data(x)
        mkm.line(data['Date'], data['Value'], color=next(cols), line_width=next(line_width), line_dash=next(line_dash), legend=desc['Description'])

mkm.legend.border_line_width = 3
mkm.legend.border_line_color = "navy"
mkm.legend.border_line_alpha = 0.5

push_notebook()
show(mkm,notebook_handle=True)